For this proyect I will scrape a stocks information page to extract all Tickers from public companies available. I will then use an API to gather 6 months of stock price history and the industry sector for each company. Whit this information I'm looking to get a list of companies with historic stock performance that I can later use. Throughout the proyect I will be saving data onto CVSs to avoid re-running the API calls and re-scraping the website.

Step 1. - Import all libraries to be used

In [127]:
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from datetime import datetime

Step 2 - Scraping https://companiesmarketcap.com/ for information about public companies. 

I will be using selenium as data needs to load prior to being able to extract it. Using requests on its own will not work.

In [299]:
driver = webdriver.Chrome()

Given the structure of the website, infomatin about companies is represented in 58 differente pages, thus I will use an array to accumulate a dataframe for each page and then concatenate all into one. 

In [289]:
dfs = []
for i in range(1,58):
    result = requests.get(f'https://companiesmarketcap.com/page/{i}', 'html5lib')
    df = pd.read_html(result.content)[0]
    dfs.append(df)

In [290]:
concatenated = pd.concat(dfs, ignore_index=True)
concatenated

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country
0,1,Apple AAPL,$2.656 T,$161.94,0.33%,NaN,🇺🇸 USA
1,2,Microsoft MSFT,$2.537 T,$337.91,0.07%,NaN,🇺🇸 USA
2,3,Alphabet (Google) GOOG,$1.943 T,"$2,934",-0.03%,NaN,🇺🇸 USA
3,4,Saudi Aramco 2222.SR,$1.896 T,$9.49,-1.11%,NaN,🇸🇦 S. Arabia
4,5,Amazon AMZN,$1.815 T,"$3,580",0.01%,NaN,🇺🇸 USA
...,...,...,...,...,...,...,...
5691,5692,Commercial International Bank CIN.F,NaN,$3.52,-1.99%,NaN,🇪🇬 Egypt
5692,5693,Sangoma Technologies SANG,NaN,NaN,0.00%,NaN,🇨🇦 Canada
5693,5694,Nykaa NYKAA.NS,NaN,NaN,0.00%,NaN,🇮🇳 India
5694,5695,Intensity Therapeutics INTS,NaN,NaN,0.00%,NaN,🇺🇸 USA


After concatenating all dataframes I will do some cleaning to obtain relevant data. The original "Name" column actually contains both the name of the company and it's "Ticker" or ("Symbol"). I will need these to be on two different columns so I can actually use the symbol on my API requests. Additionally, I will need to do some cleaning on the "Market Cap" and "Price" columns so I can convert these columns into numbers (float).

On the "Market Cap" column I notice there is a letter representing the magnitude of the number. I will also need to extract this so I can later have a "Market Cap" Column with an absolute number (not abbreviated).

In [291]:
concatenated['Ticker'] = concatenated['Name'].str.split().str[-1]

In [292]:
concatenated['Name'] = concatenated.Name.str.rsplit(' ', 1).str[0]

In [293]:
concatenated['Market Cap Mag'] = concatenated['Market Cap'].str.split().str[-1]

In [294]:
concatenated['Market Cap'] = concatenated['Market Cap'].str.rsplit(' ', 1).str[0]

In [295]:
concatenated['Market Cap'] = concatenated['Market Cap'].str.replace('$', '', regex=False)

In [296]:
to_drop = concatenated[concatenated['Market Cap'].isnull()]
concatenated.drop(to_drop.index, inplace=True)

In [297]:
concatenated['Market Cap'] = concatenated['Market Cap'].astype('float')
concatenated.dtypes

Rank                 int64
Name                object
Market Cap         float64
Price               object
Today               object
Price (30 days)    float64
Country             object
Ticker              object
Market Cap Mag      object
dtype: object

In [300]:
trillion = concatenated[concatenated['Market Cap Mag'] == 'T']
concatenated.loc[(concatenated['Market Cap Mag'] == 'T'), 'Market Cap'] = concatenated['Market Cap'] * 1000000000000
concatenated

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,Ticker,Market Cap Mag
0,1,Apple,2.656000e+24,$161.94,0.33%,NaN,🇺🇸 USA,AAPL,T
1,2,Microsoft,2.537000e+24,$337.91,0.07%,NaN,🇺🇸 USA,MSFT,T
2,3,Alphabet (Google),1.943000e+24,"$2,934",-0.03%,NaN,🇺🇸 USA,GOOG,T
3,4,Saudi Aramco,1.896000e+24,$9.49,-1.11%,NaN,🇸🇦 S. Arabia,2222.SR,T
4,5,Amazon,1.815000e+24,"$3,580",0.01%,NaN,🇺🇸 USA,AMZN,T
...,...,...,...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e-02,$2.28,-3.80%,NaN,🇺🇸 USA,TORC,B
5683,5684,Jump Networks,1.000000e-02,$0.10,-1.94%,NaN,🇮🇳 India,JUMPNET.NS,B
5684,5685,Pareteum Corporation,2.850000e+00,$0.02,-63.64%,NaN,🇺🇸 USA,TEUM,M
5685,5686,Youngevity International,1.370000e+00,$0.04,0.00%,NaN,🇺🇸 USA,YGYI,M


In [301]:
billion = concatenated[concatenated['Market Cap Mag'] == 'B']
concatenated.loc[(concatenated['Market Cap Mag'] == 'B'), 'Market Cap'] = concatenated['Market Cap'] * 1000000000
concatenated

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,Ticker,Market Cap Mag
0,1,Apple,2.656000e+24,$161.94,0.33%,NaN,🇺🇸 USA,AAPL,T
1,2,Microsoft,2.537000e+24,$337.91,0.07%,NaN,🇺🇸 USA,MSFT,T
2,3,Alphabet (Google),1.943000e+24,"$2,934",-0.03%,NaN,🇺🇸 USA,GOOG,T
3,4,Saudi Aramco,1.896000e+24,$9.49,-1.11%,NaN,🇸🇦 S. Arabia,2222.SR,T
4,5,Amazon,1.815000e+24,"$3,580",0.01%,NaN,🇺🇸 USA,AMZN,T
...,...,...,...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e+07,$2.28,-3.80%,NaN,🇺🇸 USA,TORC,B
5683,5684,Jump Networks,1.000000e+07,$0.10,-1.94%,NaN,🇮🇳 India,JUMPNET.NS,B
5684,5685,Pareteum Corporation,2.850000e+00,$0.02,-63.64%,NaN,🇺🇸 USA,TEUM,M
5685,5686,Youngevity International,1.370000e+00,$0.04,0.00%,NaN,🇺🇸 USA,YGYI,M


In [302]:
million = concatenated[concatenated['Market Cap Mag'] == 'M']
concatenated.loc[(concatenated['Market Cap Mag'] == 'M'), 'Market Cap'] = concatenated['Market Cap'] * 1000000
concatenated

,Rank,Name,Market Cap,Price,Today,Price (30 days),Country,Ticker,Market Cap Mag
0,1,Apple,2.656000e+24,$161.94,0.33%,NaN,🇺🇸 USA,AAPL,T
1,2,Microsoft,2.537000e+24,$337.91,0.07%,NaN,🇺🇸 USA,MSFT,T
2,3,Alphabet (Google),1.943000e+24,"$2,934",-0.03%,NaN,🇺🇸 USA,GOOG,T
3,4,Saudi Aramco,1.896000e+24,$9.49,-1.11%,NaN,🇸🇦 S. Arabia,2222.SR,T
4,5,Amazon,1.815000e+24,"$3,580",0.01%,NaN,🇺🇸 USA,AMZN,T
...,...,...,...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e+07,$2.28,-3.80%,NaN,🇺🇸 USA,TORC,B
5683,5684,Jump Networks,1.000000e+07,$0.10,-1.94%,NaN,🇮🇳 India,JUMPNET.NS,B
5684,5685,Pareteum Corporation,2.850000e+06,$0.02,-63.64%,NaN,🇺🇸 USA,TEUM,M
5685,5686,Youngevity International,1.370000e+06,$0.04,0.00%,NaN,🇺🇸 USA,YGYI,M


In [303]:
concatenated.drop(['Price (30 days)', 'Market Cap Mag', 'Today'], axis=1, inplace= True)


In [304]:
concatenated['Price'] = concatenated.Price.str.replace('[$,]', '', regex=True)

In [305]:
concatenated

,Rank,Name,Market Cap,Price,Country,Ticker
0,1,Apple,2.656000e+24,161.94,🇺🇸 USA,AAPL
1,2,Microsoft,2.537000e+24,337.91,🇺🇸 USA,MSFT
2,3,Alphabet (Google),1.943000e+24,2934,🇺🇸 USA,GOOG
3,4,Saudi Aramco,1.896000e+24,9.49,🇸🇦 S. Arabia,2222.SR
4,5,Amazon,1.815000e+24,3580,🇺🇸 USA,AMZN
...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e+07,2.28,🇺🇸 USA,TORC
5683,5684,Jump Networks,1.000000e+07,0.10,🇮🇳 India,JUMPNET.NS
5684,5685,Pareteum Corporation,2.850000e+06,0.02,🇺🇸 USA,TEUM
5685,5686,Youngevity International,1.370000e+06,0.04,🇺🇸 USA,YGYI


In [306]:
concatenated['Price'] = concatenated['Price'].astype('float')
concatenated.dtypes

Rank            int64
Name           object
Market Cap    float64
Price         float64
Country        object
Ticker         object
dtype: object

In [307]:
concatenated['Shares'] = concatenated['Market Cap'] / concatenated['Price']

In [308]:
concatenated

,Rank,Name,Market Cap,Price,Country,Ticker,Shares
0,1,Apple,2.656000e+24,161.94,🇺🇸 USA,AAPL,1.640114e+22
1,2,Microsoft,2.537000e+24,337.91,🇺🇸 USA,MSFT,7.507916e+21
2,3,Alphabet (Google),1.943000e+24,2934.00,🇺🇸 USA,GOOG,6.622359e+20
3,4,Saudi Aramco,1.896000e+24,9.49,🇸🇦 S. Arabia,2222.SR,1.997893e+23
4,5,Amazon,1.815000e+24,3580.00,🇺🇸 USA,AMZN,5.069832e+20
...,...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e+07,2.28,🇺🇸 USA,TORC,4.385965e+06
5683,5684,Jump Networks,1.000000e+07,0.10,🇮🇳 India,JUMPNET.NS,1.000000e+08
5684,5685,Pareteum Corporation,2.850000e+06,0.02,🇺🇸 USA,TEUM,1.425000e+08
5685,5686,Youngevity International,1.370000e+06,0.04,🇺🇸 USA,YGYI,3.425000e+07


After cleaning, creating new columuns, and dropping unnecesary columns. I save this as a CSV.

In [309]:
concatenated.to_csv('./output/stocks.csv', index=False)

In [224]:
concatenated = pd.read_csv('./output/stocks.csv')

,Rank,Name,Market Cap,Price,Country,Ticker,Shares
0,1,Apple,2.656000e+24,161.94,🇺🇸 USA,AAPL,1.640114e+22
1,2,Microsoft,2.537000e+24,337.91,🇺🇸 USA,MSFT,7.507916e+21
2,3,Alphabet (Google),1.943000e+24,2934.00,🇺🇸 USA,GOOG,6.622359e+20
3,4,Saudi Aramco,1.896000e+24,9.49,🇸🇦 S. Arabia,2222.SR,1.997893e+23
4,5,Amazon,1.815000e+24,3580.00,🇺🇸 USA,AMZN,5.069832e+20
...,...,...,...,...,...,...,...
5682,5683,resTORbio,1.000000e+07,2.28,🇺🇸 USA,TORC,4.385965e+06
5683,5684,Jump Networks,1.000000e+07,0.10,🇮🇳 India,JUMPNET.NS,1.000000e+08
5684,5685,Pareteum Corporation,2.850000e+06,0.02,🇺🇸 USA,TEUM,1.425000e+08
5685,5686,Youngevity International,1.370000e+06,0.04,🇺🇸 USA,YGYI,3.425000e+07


Step 3 - Complementing information with API. 

I will take all symbols or Tickers from the scraped data and feed these into an API so I can get historic stock price data.

The API I will be using is: yahoofinanceapi.com

In [107]:
key = os.getenv('yfapi_token')

In [213]:
tickers = concatenated.Ticker.to_list()
tickers

['AAPL',
 'MSFT',
 'GOOG',
 '2222.SR',
 'AMZN',
 'TSLA',
 'FB',
 'NVDA',
 'BRK-A',
 'TSM',
 'TCEHY',
 'JPM',
 'V',
 'HD',
 'UNH',
 'JNJ',
 'LVMUY',
 'WMT',
 '005930.KS',
 'BAC',
 '600519.SS',
 'BABA',
 'PG',
 'NSRGY',
 'RHHBY',
 'MA',
 'ASML',
 'ADBE',
 'NFLX',
 'PFE',
 'CRM',
 'DIS',
 'NKE',
 'XOM',
 'OR.PA',
 'TM',
 'ORCL',
 'LLY',
 'TMO',
 'NVO',
 'COST',
 '300750.SZ',
 '1398.HK',
 'KO',
 'CMCSA',
 'CSCO',
 'AVGO',
 'ACN',
 'PEP',
 'CVX',
 'DHR',
 'PYPL',
 'ABT',
 'RELIANCE.NS',
 'MPNGF',
 'VZ',
 'ABBV',
 'MRK',
 'WFC',
 'SHOP',
 'QCOM',
 'INTC',
 '3968.HK',
 'INTU',
 'HESAF',
 'MCD',
 'AMD',
 'NVS',
 'MS',
 'UPS',
 'TXN',
 'AZN',
 'T',
 'LOW',
 'RYDAF',
 'NEE',
 'PRX.VI',
 'TCS.NS',
 'CICHY',
 'LIN',
 'SE',
 'UNP',
 'SAP',
 'SCHW',
 'ACGBY',
 'MDT',
 'KYCCF',
 'SONY',
 'RY',
 'HON',
 'CDI.PA',
 'TMUS',
 'BLK',
 'JD',
 'BHP',
 'PM',
 'PNGAY',
 'TD',
 'SIEGY',
 'AAIGF',
 'PTR',
 'AMAT',
 '000858.SZ',
 'UL',
 'SBUX',
 'C',
 'GS',
 '002594.SZ',
 'AXP',
 'RTX',
 'BACHF',
 'NOW',
 'VOW3.

For the API calls I will be using "time" to ensure I do not exceed the 300 calls/minute threshold and will be using try and except in case a symbol is not included in the API.

In [312]:
dfs_fin_hist = []
for symbol in tickers:
    time.sleep(0.22)
    try:
        response = requests.get(f"https://yfapi.net/v8/finance/spark?interval=1d&range=6mo&symbols={symbol}", headers={'x-api-key': key})
        res = response.json()
        pd.json_normalize(res[symbol])
        times= [datetime.fromtimestamp(a).strftime('%Y-%m-%d') for a in res[symbol]['timestamp']]
        data = {'Ticker': [res[symbol]['symbol']]}
        close_data = [list(x) for x in zip(times, res[symbol]['close'])]
        for i in close_data:
            data[i[0]] = [i[1]]
        df_fin_hist = pd.DataFrame(data)
        dfs_fin_hist.append(df_fin_hist)
    except:
        pass


KeyboardInterrupt: 

In [12]:
dfs_fin_hist_concat = pd.concat(dfs_fin_hist, ignore_index=True, sort=False)


NameError: name 'dfs_fin_hist' is not defined

In [13]:
dfs_fin_hist_concat.to_csv('./output/historic_stock_prices.csv')

NameError: name 'dfs_final' is not defined

In [27]:
dfs_final = pd.read_csv('./output/historic_stock_prices.csv')
dfs_final.drop(['Unnamed: 0'], axis=1, inplace=True)
dfs_final

,2021-05-23,2021-05-24,2021-05-25,2021-05-26,2021-05-27,2021-05-28,2021-05-30,2021-05-31,2021-06-01,2021-06-02,...,2021-11-15,2021-11-16,2021-11-17,2021-11-18,2021-11-19,2021-11-21,2021-11-22,2021-11-23,2021-11-24,Ticker
0,NaN,127.10,126.90,126.850,125.28,124.610,NaN,NaN,124.280,125.06,...,150.0000,151.00,153.49,157.87,160.55,NaN,161.0200,161.4100,NaN,AAPL
1,NaN,250.78,251.72,251.490,249.31,249.680,NaN,NaN,247.400,247.30,...,336.0700,339.51,339.12,341.27,343.11,NaN,339.8300,337.6800,NaN,MSFT
2,NaN,2406.67,2409.07,2433.530,2402.51,2411.560,NaN,NaN,2429.810,2421.28,...,2987.7600,2981.52,2981.24,3014.18,2999.05,NaN,2941.5700,2935.1400,NaN,GOOG
3,35.45,35.35,35.40,35.300,35.25,NaN,35.30,35.3,35.400,35.40,...,37.2500,37.15,37.05,37.05,NaN,36.10,35.4500,35.8000,NaN,2222.SR
4,NaN,3244.99,3259.05,3265.160,3230.11,3223.070,NaN,NaN,3218.650,3233.99,...,3545.6800,3540.70,3549.00,3696.06,3676.57,NaN,3572.5700,3580.0400,NaN,AMZN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5534,NaN,3.86,3.91,4.070,4.06,4.140,NaN,NaN,4.240,4.29,...,2.6300,2.57,2.53,2.43,2.40,NaN,2.2800,2.3300,NaN,EFOI
5535,6.90,7.20,7.20,7.050,6.95,NaN,7.25,NaN,7.950,8.30,...,7.7500,7.80,7.75,NaN,NaN,7.55,7.9000,NaN,7.65,JUMPNET.NS
5536,NaN,0.37,0.36,0.370,0.36,0.350,NaN,NaN,0.350,0.34,...,0.0700,0.06,NaN,0.06,0.05,NaN,0.0500,0.0600,NaN,TEUM
5537,NaN,0.32,0.34,0.355,0.32,0.335,NaN,NaN,0.310,0.33,...,0.0551,NaN,NaN,NaN,NaN,NaN,0.0551,0.0404,NaN,YGYI


After my API calls are done and I have saved the information in a csv. I proceed to merge both files (1. the scraped data and 2. The data extracted from the API)

In [225]:
merged = pd.merge(concatenated, dfs_final, how="inner", on=["Ticker", "Ticker"])
merged.to_csv('./output/merged.csv')

Looking at the data, It's becoming a bit complicated to get anything meaningfull on the historic prices. Actually what I want to know is what stocks are the big winners, and big loosers. Eventually understand any correlations between these... i.e. is there a stock that when its a big winner other stock is a big looser?

As a starting point I want to understand both absolute and % increase/decrease of stock price in the last 6 months. Note: API data includes 6 months history. 
To do this I will use melt to pivot date columns into a single column. This will allow me to more easily perform analyses on stock prices.

In [29]:
merged_melt = pd.melt(merged, id_vars=['Rank', 'Name', 'Market Cap', 'Price', 'Country', 'Ticker', 'Shares'])
merged_melt['Converted Date'] = pd.to_datetime(merged_melt['variable'], format="%Y-%m-%d")
merged_melt['Year'] = pd.DatetimeIndex(merged_melt['Converted Date']).year
merged_melt['Month'] = pd.DatetimeIndex(merged_melt['Converted Date']).month
merged_melt['Day'] = pd.DatetimeIndex(merged_melt['Converted Date']).day
merged_melt.drop(['Rank', 'Market Cap', 'Price', 'Country', 'Shares', 'variable'], axis=1, inplace=True)
merged_melt.rename(columns = {'value': 'Stock Price', 'Converted Date': 'Date'}, inplace = True)
merged_melt

,Name,Ticker,Stock Price,Date,Year,Month,Day
0,Apple,AAPL,NaN,2021-05-23,2021,5,23
1,Microsoft,MSFT,NaN,2021-05-23,2021,5,23
2,Alphabet (Google),GOOG,NaN,2021-05-23,2021,5,23
3,Saudi Aramco,2222.SR,35.45,2021-05-23,2021,5,23
4,Amazon,AMZN,NaN,2021-05-23,2021,5,23
...,...,...,...,...,...,...,...
886235,Energy Focus,EFOI,NaN,2021-11-24,2021,11,24
886236,Jump Networks,JUMPNET.NS,7.65,2021-11-24,2021,11,24
886237,Pareteum Corporation,TEUM,NaN,2021-11-24,2021,11,24
886238,Youngevity International,YGYI,NaN,2021-11-24,2021,11,24


After melting I will sort to make sure information is displayed with date ascending. 

In [30]:
merged_melt_wo_nulls = merged_melt[merged_melt['Stock Price'].isnull() == False]
merged_melt_wo_nulls = merged_melt_wo_nulls.sort_values(['Ticker', 'Date'], ascending=[True, True])

In [31]:
merged_melt_wo_nulls['Stock Price'] = merged_melt_wo_nulls['Stock Price'].astype('float')
merged_melt_wo_nulls.dtypes

Name                   object
Ticker                 object
Stock Price           float64
Date           datetime64[ns]
Year                    int64
Month                   int64
Day                     int64
dtype: object

Step 4 - Applying formulas to data so we can get the 6-month-gain (absolute and percent) and a function that can get the gain between any two differente months.

In [125]:
list_of_tickers = list(merged_melt_wo_nulls.Ticker.unique())
len(list_of_tickers)

5539

In [159]:
merged['6mo Gain Abs'] = 0
merged['6mo Gain Pct'] = 0
merged['1st Month Gain'] = 0
merged['6mo Gain Abs'] = merged['6mo Gain Abs'].astype('float')
merged['6mo Gain Pct'] = merged['6mo Gain Pct'].astype('float')
merged['1st Month Gain'] = merged['1st Month Gain'].astype('float')
merged.dtypes

Rank                int64
Name               object
Market Cap        float64
Price             float64
Country            object
                   ...   
2021-11-23        float64
2021-11-24        float64
6mo Gain Abs      float64
6mo Gain Pct      float64
1st Month Gain    float64
Length: 170, dtype: object

In [160]:
merged.set_index('Ticker', inplace=True)

In [161]:
merged

,Rank,Name,Market Cap,Price,Country,Shares,2021-05-23,2021-05-24,2021-05-25,2021-05-26,...,2021-11-17,2021-11-18,2021-11-19,2021-11-21,2021-11-22,2021-11-23,2021-11-24,6mo Gain Abs,6mo Gain Pct,1st Month Gain
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,1,Apple,2.656000e+24,161.94,🇺🇸 USA,1.640114e+22,NaN,127.10,126.90,126.850,...,153.49,157.87,160.55,NaN,161.0200,161.4100,NaN,0.0,0.0,0.0
MSFT,2,Microsoft,2.537000e+24,337.91,🇺🇸 USA,7.507916e+21,NaN,250.78,251.72,251.490,...,339.12,341.27,343.11,NaN,339.8300,337.6800,NaN,0.0,0.0,0.0
GOOG,3,Alphabet (Google),1.943000e+24,2934.00,🇺🇸 USA,6.622359e+20,NaN,2406.67,2409.07,2433.530,...,2981.24,3014.18,2999.05,NaN,2941.5700,2935.1400,NaN,0.0,0.0,0.0
2222.SR,4,Saudi Aramco,1.896000e+24,9.49,🇸🇦 S. Arabia,1.997893e+23,35.45,35.35,35.40,35.300,...,37.05,37.05,NaN,36.10,35.4500,35.8000,NaN,0.0,0.0,0.0
AMZN,5,Amazon,1.815000e+24,3580.00,🇺🇸 USA,5.069832e+20,NaN,3244.99,3259.05,3265.160,...,3549.00,3696.06,3676.57,NaN,3572.5700,3580.0400,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EFOI,5682,Energy Focus,1.000000e+07,2.50,🇺🇸 USA,4.000000e+06,NaN,3.86,3.91,4.070,...,2.53,2.43,2.40,NaN,2.2800,2.3300,NaN,0.0,0.0,0.0
JUMPNET.NS,5684,Jump Networks,1.000000e+07,0.10,🇮🇳 India,1.000000e+08,6.90,7.20,7.20,7.050,...,7.75,NaN,NaN,7.55,7.9000,NaN,7.65,0.0,0.0,0.0
TEUM,5685,Pareteum Corporation,2.850000e+06,0.02,🇺🇸 USA,1.425000e+08,NaN,0.37,0.36,0.370,...,NaN,0.06,0.05,NaN,0.0500,0.0600,NaN,0.0,0.0,0.0


In [162]:
def six_month_gain_absolute(tickers):
    for ticker in tickers:
        comp = merged_melt_wo_nulls[merged_melt_wo_nulls['Ticker'] == ticker]
        a = comp.iloc[-1]['Stock Price']-comp.iloc[0]['Stock Price']
        merged.at[ticker, '6mo Gain Abs'] = a
    return a
def six_month_gain_percent(tickers):
    for ticker in tickers:
        comp = merged_melt_wo_nulls[merged_melt_wo_nulls['Ticker'] == ticker]
        a_percent = comp.iloc[-1]['Stock Price']/comp.iloc[0]['Stock Price'] - 1
        merged.at[ticker, '6mo Gain Pct'] = a_percent
    
def monthly_period_gain(tickers, month_1, month_2):
    for ticker in tickers:
        try:
            comp_m1 = merged_melt_wo_nulls[(merged_melt_wo_nulls['Ticker'] == ticker) & (merged_melt_wo_nulls['Month'] == month_1) ]
            comp_m2 = merged_melt_wo_nulls[(merged_melt_wo_nulls['Ticker'] == ticker) & (merged_melt_wo_nulls['Month'] == month_2) ]
            a = comp_m2.iloc[-1]['Stock Price']-comp_m1.iloc[-1]['Stock Price']
            merged.at[ticker, '1st Month Gain'] = a
        except:
            pass
    

In [163]:
six_month_gain_absolute(list_of_tickers)
six_month_gain_percent(list_of_tickers)
monthly_period_gain(list_of_tickers, 5, 6)
merged

,Rank,Name,Market Cap,Price,Country,Shares,2021-05-23,2021-05-24,2021-05-25,2021-05-26,...,2021-11-17,2021-11-18,2021-11-19,2021-11-21,2021-11-22,2021-11-23,2021-11-24,6mo Gain Abs,6mo Gain Pct,1st Month Gain
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,1,Apple,2.656000e+24,161.94,🇺🇸 USA,1.640114e+22,NaN,127.10,126.90,126.850,...,153.49,157.87,160.55,NaN,161.0200,161.4100,NaN,34.3100,0.269945,12.350
MSFT,2,Microsoft,2.537000e+24,337.91,🇺🇸 USA,7.507916e+21,NaN,250.78,251.72,251.490,...,339.12,341.27,343.11,NaN,339.8300,337.6800,NaN,86.9000,0.346519,21.220
GOOG,3,Alphabet (Google),1.943000e+24,2934.00,🇺🇸 USA,6.622359e+20,NaN,2406.67,2409.07,2433.530,...,2981.24,3014.18,2999.05,NaN,2941.5700,2935.1400,NaN,528.4700,0.219586,94.760
2222.SR,4,Saudi Aramco,1.896000e+24,9.49,🇸🇦 S. Arabia,1.997893e+23,35.45,35.35,35.40,35.300,...,37.05,37.05,NaN,36.10,35.4500,35.8000,NaN,0.3500,0.009873,-0.200
AMZN,5,Amazon,1.815000e+24,3580.00,🇺🇸 USA,5.069832e+20,NaN,3244.99,3259.05,3265.160,...,3549.00,3696.06,3676.57,NaN,3572.5700,3580.0400,NaN,335.0500,0.103251,217.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EFOI,5682,Energy Focus,1.000000e+07,2.50,🇺🇸 USA,4.000000e+06,NaN,3.86,3.91,4.070,...,2.53,2.43,2.40,NaN,2.2800,2.3300,NaN,-1.5300,-0.396373,-0.160
JUMPNET.NS,5684,Jump Networks,1.000000e+07,0.10,🇮🇳 India,1.000000e+08,6.90,7.20,7.20,7.050,...,7.75,NaN,NaN,7.55,7.9000,NaN,7.65,0.7500,0.108696,3.900
TEUM,5685,Pareteum Corporation,2.850000e+06,0.02,🇺🇸 USA,1.425000e+08,NaN,0.37,0.36,0.370,...,NaN,0.06,0.05,NaN,0.0500,0.0600,NaN,-0.3100,-0.837838,0.020


In [166]:
merged.to_csv('./output/merged_with_formulas.csv')

After applying formulas and given they do take a while to run, I save the updated dataframe in a new file.

Step 5 - Extracting sector data from API. While individual stock analyses is valuable, sometimes whole industries are affected or might be correlated, thus I want to add the industry sector for each stock.

In [144]:
dfs_industry = []
for symbol in list_of_tickers:
    time.sleep(0.22)
    try:
        response = requests.get(f"https://yfapi.net/ws/insights/v1/finance/insights?symbol={symbol}", headers={'x-api-key': key})
        res = response.json()
        dfs_industry.append([symbol, res['finance']['result']['companySnapshot']['sectorInfo']])
    except:
        pass
sectors = pd.DataFrame(dfs_industry, columns=['Ticker', 'Sector'])
sectors

,Ticker,Sector
0,A,Healthcare
1,AA,Basic Materials
2,AAIGF,Financial Services
3,AAL,Industrials
4,AAN,Consumer Cyclical
...,...,...
3669,ZVIA,Consumer Defensive
3670,ZVO,Consumer Defensive
3671,ZYME,Healthcare
3672,ZYNE,Healthcare


In [192]:
sectors.to_csv('./output/sectors.csv', index=False)

In [194]:
sectors = pd.read_csv('./output/sectors.csv').drop(['Unnamed: 0'],axis=1)

In [195]:
sectors

,Ticker,Sector
0,A,Healthcare
1,AA,Basic Materials
2,AAIGF,Financial Services
3,AAL,Industrials
4,AAN,Consumer Cyclical
...,...,...
3669,ZVIA,Consumer Defensive
3670,ZVO,Consumer Defensive
3671,ZYME,Healthcare
3672,ZYNE,Healthcare


In [196]:
merged_new = pd.merge(merged, sectors, how="left", on=["Ticker", "Ticker"])

In [222]:
merged_new.to_csv('./output/full_integration.csv', index=False)

Up until this point it seems we have a pretty comprehensive data base of publick stock, nonetheless it still needs some transformations...

Step 6 - Transformations (qcut + dummies)

In [215]:
base = pd.read_csv('full_integration.csv')

In [216]:
clean_base = base[['Ticker', 'Rank', 'Name', 'Market Cap', 'Country', 'Shares', '6mo Gain Abs', '6mo Gain Pct', '1st Month Gain', 'Sector']]

In [217]:
clean_base.shape

(5539, 10)

In [204]:
clean_base.dtypes

Ticker             object
Rank                int64
Name               object
Market Cap        float64
Country            object
Shares            float64
6mo Gain Abs      float64
6mo Gain Pct      float64
1st Month Gain    float64
Sector             object
dtype: object

In [205]:
clean_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5539 entries, 0 to 5538
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ticker          5539 non-null   object 
 1   Rank            5539 non-null   int64  
 2   Name            5539 non-null   object 
 3   Market Cap      5539 non-null   float64
 4   Country         5539 non-null   object 
 5   Shares          5538 non-null   float64
 6   6mo Gain Abs    5539 non-null   float64
 7   6mo Gain Pct    5539 non-null   float64
 8   1st Month Gain  5539 non-null   float64
 9   Sector          3674 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 432.9+ KB


In [206]:
clean_base.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Ticker,5539,5539,AAPL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rank,5539.0,NaN,NaN,NaN,2808.790937,1636.958619,1.0,1392.5,2789.0,4220.5,5687.0
Name,5539,5536,Cheniere Energy,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Market Cap,5539.0,NaN,NaN,NaN,2160498284904356773888.0,67701707393579396104192.0,0.0,1010000000.0,3920000000.0,12530000000.0,2656000000000000067108864.0
Country,5539,64,🇺🇸 USA,2993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shares,5538.0,NaN,NaN,NaN,40785682143138439168.0,2695610840969511960576.0,106842.142503,49160086.184656,128514617.506906,415107389.817888,199789251844046399209472.0
6mo Gain Abs,5539.0,NaN,NaN,NaN,-102.315928,4090.632959,-148000.0,-3.28,0.2,8.86,54000.0
6mo Gain Pct,5539.0,NaN,NaN,NaN,0.03927,0.357158,-0.948895,-0.136435,0.011527,0.166855,7.247664
1st Month Gain,5539.0,NaN,NaN,NaN,44.965858,1821.433646,-38000.0,-1.47,0.0,2.64,74000.0
Sector,3674,11,Healthcare,660,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
clean_base.describe(percentiles=[x/10 for x in range(10)]).T

,count,mean,std,min,0%,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
Rank,5539.0,2.808791e+03,1.636959e+03,1.000000,1.000000,5.548000e+02,1.110600e+03,1.670400e+03,2.228200e+03,2.789000e+03,3.360800e+03,3.932600e+03,4.509400e+03,5.090200e+03,5.687000e+03
Market Cap,5539.0,2.160498e+21,6.770171e+22,0.000000,0.000000,2.500000e+08,6.800000e+08,1.400000e+09,2.432000e+09,3.920000e+09,5.900000e+09,9.496000e+09,1.682400e+10,3.619800e+10,2.656000e+24
Shares,5538.0,4.078568e+19,2.695611e+21,106842.142503,106842.142503,2.232322e+07,3.931914e+07,5.857185e+07,8.846897e+07,1.285146e+08,1.888337e+08,2.971794e+08,5.974936e+08,1.800650e+09,1.997893e+23
6mo Gain Abs,5539.0,-1.023159e+02,4.090633e+03,-148000.000000,-148000.000000,-1.373400e+01,-5.140000e+00,-2.090000e+00,-6.600000e-01,2.000000e-01,1.737600e+00,5.266000e+00,1.469000e+01,5.517800e+01,5.400000e+04
6mo Gain Pct,5539.0,3.926972e-02,3.571577e-01,-0.948895,-0.948895,-2.984984e-01,-1.727584e-01,-1.014437e-01,-4.396817e-02,1.152738e-02,6.146270e-02,1.256341e-01,2.182987e-01,3.725370e-01,7.247664e+00
1st Month Gain,5539.0,4.496586e+01,1.821434e+03,-38000.000000,-38000.000000,-5.100000e+00,-2.130000e+00,-9.600000e-01,-2.800000e-01,0.000000e+00,4.100000e-01,1.526000e+00,4.464000e+00,1.700000e+01,7.400000e+04


In [219]:
tags = ['big loss', 'loss', 'neutral', 'gain', 'big gain']
clean_base['6mo Veredict'] = pd.qcut(clean_base['6mo Gain Pct'], 5, labels=tags)
clean_base

/var/folders/bg/fmcp93r90mzd7b67zml5jgwm0000gn/T/ipykernel_47067/3913190481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_base['6mo Veredict'] = pd.qcut(clean_base['6mo Gain Pct'], 5, labels=tags)


,Ticker,Rank,Name,Market Cap,Country,Shares,6mo Gain Abs,6mo Gain Pct,1st Month Gain,Sector,6mo Veredict
0,AAPL,1,Apple,2.656000e+24,🇺🇸 USA,1.640114e+22,34.3100,0.269945,12.350,Technology,big gain
1,MSFT,2,Microsoft,2.537000e+24,🇺🇸 USA,7.507916e+21,86.9000,0.346519,21.220,Technology,big gain
2,GOOG,3,Alphabet (Google),1.943000e+24,🇺🇸 USA,6.622359e+20,528.4700,0.219586,94.760,Communication Services,big gain
3,2222.SR,4,Saudi Aramco,1.896000e+24,🇸🇦 S. Arabia,1.997893e+23,0.3500,0.009873,-0.200,NaN,neutral
4,AMZN,5,Amazon,1.815000e+24,🇺🇸 USA,5.069832e+20,335.0500,0.103251,217.090,Consumer Cyclical,gain
...,...,...,...,...,...,...,...,...,...,...,...
5534,EFOI,5682,Energy Focus,1.000000e+07,🇺🇸 USA,4.000000e+06,-1.5300,-0.396373,-0.160,Consumer Cyclical,big loss
5535,JUMPNET.NS,5684,Jump Networks,1.000000e+07,🇮🇳 India,1.000000e+08,0.7500,0.108696,3.900,NaN,gain
5536,TEUM,5685,Pareteum Corporation,2.850000e+06,🇺🇸 USA,1.425000e+08,-0.3100,-0.837838,0.020,Technology,big loss
5537,YGYI,5686,Youngevity International,1.370000e+06,🇺🇸 USA,3.425000e+07,-0.2796,-0.873750,-0.025,Consumer Defensive,big loss


In [220]:
df_dummies = pd.get_dummies(clean_base,drop_first=True,columns=['Sector','6mo Veredict'])
df_dummies

,Ticker,Rank,Name,Market Cap,Country,Shares,6mo Gain Abs,6mo Gain Pct,1st Month Gain,Sector_Communication Services,...,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,Sector_Real Estate,Sector_Technology,Sector_Utilities,6mo Veredict_loss,6mo Veredict_neutral,6mo Veredict_gain,6mo Veredict_big gain
0,AAPL,1,Apple,2.656000e+24,🇺🇸 USA,1.640114e+22,34.3100,0.269945,12.350,0,...,0,0,0,0,1,0,0,0,0,1
1,MSFT,2,Microsoft,2.537000e+24,🇺🇸 USA,7.507916e+21,86.9000,0.346519,21.220,0,...,0,0,0,0,1,0,0,0,0,1
2,GOOG,3,Alphabet (Google),1.943000e+24,🇺🇸 USA,6.622359e+20,528.4700,0.219586,94.760,1,...,0,0,0,0,0,0,0,0,0,1
3,2222.SR,4,Saudi Aramco,1.896000e+24,🇸🇦 S. Arabia,1.997893e+23,0.3500,0.009873,-0.200,0,...,0,0,0,0,0,0,0,1,0,0
4,AMZN,5,Amazon,1.815000e+24,🇺🇸 USA,5.069832e+20,335.0500,0.103251,217.090,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5534,EFOI,5682,Energy Focus,1.000000e+07,🇺🇸 USA,4.000000e+06,-1.5300,-0.396373,-0.160,0,...,0,0,0,0,0,0,0,0,0,0
5535,JUMPNET.NS,5684,Jump Networks,1.000000e+07,🇮🇳 India,1.000000e+08,0.7500,0.108696,3.900,0,...,0,0,0,0,0,0,0,0,1,0
5536,TEUM,5685,Pareteum Corporation,2.850000e+06,🇺🇸 USA,1.425000e+08,-0.3100,-0.837838,0.020,0,...,0,0,0,0,1,0,0,0,0,0
5537,YGYI,5686,Youngevity International,1.370000e+06,🇺🇸 USA,3.425000e+07,-0.2796,-0.873750,-0.025,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
df_dummies.to_csv('./output/final_extract.csv', index=False)